In [1]:
import pandas as pd
import ntpath
import torchvision.transforms as transforms
from imgaug import augmenters as iaa
import os
import numpy as np
from PIL import Image
import matplotlib.image as mpimg

class dataAugmentation:
    def __init__(self):
        pass

    def horizontal_flipping(self, img):
        Horizontal_Flipping_Transformation = transforms.RandomHorizontalFlip(p=1) 
        flipping_img = Horizontal_Flipping_Transformation(img)
        return flipping_img
    
    def brightness_cambiar(self, img):
        brightness = transforms.ColorJitter(brightness=(0.4, 1))
        changed_img = brightness(img)
        return changed_img
    
    def shift(self, img):
        pan = iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)})
        image = pan.augment_image(np.asarray(img))
        image = Image.fromarray(np.uint8(image))
        return image
    
    def blur(self, img):
        transf = transforms.GaussianBlur( (5,9), sigma=(3,5))
        img=transf(img)
        return img

**Data augmentation for sequential model**

In [ ]:
import re
import os
from PIL import Image
import numpy as np
ROOT_FOLDER_IMAGES='trainsetRNN/new_images/new_imageSequence'
ROOT_FOLDER_MASKS='trainsetRNN/new_images/new_maskSequence'

list=[]
typeAug = ["Flipped", "BrightnessChange", "Blurred"]
Sequences_dict={"Images":[], "Mask_output":[]}
dataAug = dataAugmentation()
for folder_typeOfDataAug in set(os.listdir(ROOT_FOLDER_IMAGES))-{'desktop.ini'}:
    if folder_typeOfDataAug=="original":
        for folder_videoCaptured in set(os.listdir(os.path.join(ROOT_FOLDER_IMAGES,folder_typeOfDataAug)))-{'desktop.ini'}:
            for folder_sequence in set(os.listdir(os.path.join(ROOT_FOLDER_IMAGES,folder_typeOfDataAug,folder_videoCaptured)))-{'desktop.ini'}:
                folder_images_path = os.path.join(ROOT_FOLDER_IMAGES,folder_typeOfDataAug,folder_videoCaptured, folder_sequence)
                ordered = os.listdir(os.path.join(folder_images_path))
                image_list = [os.path.join(folder_images_path, image) for image in ordered if image!="desktop.ini"][-5:]

                folder_masks_path = os.path.join(ROOT_FOLDER_MASKS,folder_typeOfDataAug,folder_videoCaptured)
                mask_filename = folder_sequence+".png"
                mask_path = os.path.join(folder_masks_path, mask_filename)
                mask = Image.open(mask_path)

                NumDataAug = np.random.randint(0,3)
                if NumDataAug==0:
                    MaskAfterAug = dataAug.horizontal_flipping(mask)
                else: 
                    MaskAfterAug = mask
                
                output_mask_dir = os.path.join(ROOT_FOLDER_MASKS,typeAug[NumDataAug],folder_videoCaptured)
                output_imgSequence_dir = os.path.join(ROOT_FOLDER_IMAGES,typeAug[NumDataAug],folder_videoCaptured, folder_sequence)
                if not os.path.exists(output_mask_dir):
                    # Create the folder
                    os.makedirs(output_mask_dir)
                    print(f"Folder created at: {output_mask_dir}")
                else:
                    print("Folder already exists.")

                if not os.path.exists(output_imgSequence_dir):
                    # Create the folder
                    os.makedirs(output_imgSequence_dir)
                    print(f"Folder created at: {output_imgSequence_dir}")
                else:
                    print("Folder already exists.")

                MaskAfterAugPath = os.path.join(output_mask_dir, os.path.basename(mask_path))
                MaskAfterAug.save(MaskAfterAugPath)

                for image_path in image_list:
                    img = Image.open(image_path)

                    if NumDataAug==0:
                        imgAfterAug = dataAug.horizontal_flipping(img)
                    elif NumDataAug==1:
                        imgAfterAug = dataAug.brightness_cambiar(img)
                    elif NumDataAug==2:
                        imgAfterAug = dataAug.blur(img)

                        
                    imgAfterAugPath = os.path.join(output_imgSequence_dir, os.path.basename(image_path))
                    imgAfterAug.save(imgAfterAugPath)


**Data augmentation for UNET**

In [2]:
import re
import os
from PIL import Image
import numpy as np
ROOT_FOLDER_IMAGES='trainsetRNN/image_copy'
ROOT_FOLDER_MASKS='trainsetRNN/truthWithoutSequences'

list=[]
typeAug = ["Flipped", "BrightnessChange", "Blurred"]
Sequences_dict={"Images":[], "Mask_output":[]}
dataAug = dataAugmentation()
for folder_typeOfDataAug in set(os.listdir(ROOT_FOLDER_IMAGES))-{'desktop.ini'}:
    if folder_typeOfDataAug=="original":
        for folder_videoCaptured in set(os.listdir(os.path.join(ROOT_FOLDER_IMAGES,folder_typeOfDataAug)))-{'desktop.ini'}:
            folder_images_path = os.path.join(ROOT_FOLDER_IMAGES,folder_typeOfDataAug,folder_videoCaptured)
            ordered = os.listdir(os.path.join(folder_images_path))
            image_list = [os.path.join(folder_images_path, image) for image in ordered if image!="desktop.ini"]

            folder_masks_path = os.path.join(ROOT_FOLDER_MASKS,folder_typeOfDataAug,folder_videoCaptured)

            for image_path in image_list:
                NumDataAug = np.random.randint(0,3)
                
                output_imgSequence_dir = os.path.join(ROOT_FOLDER_IMAGES,typeAug[NumDataAug],folder_videoCaptured)
                output_mask_dir = os.path.join(ROOT_FOLDER_MASKS,typeAug[NumDataAug],folder_videoCaptured)

                if not os.path.exists(output_mask_dir):
                    # Create the folder
                    os.makedirs(output_mask_dir)
                    print(f"Folder created at: {output_mask_dir}")
                else:
                    print("Folder already exists.")

                if not os.path.exists(output_imgSequence_dir):
                    # Create the folder
                    os.makedirs(output_imgSequence_dir)
                    print(f"Folder created at: {output_imgSequence_dir}")
                else:
                    print("Folder already exists.")
                
                img = Image.open(image_path)

                if NumDataAug==0:
                    imgAfterAug = dataAug.horizontal_flipping(img)
                elif NumDataAug==1:
                    imgAfterAug = dataAug.brightness_cambiar(img)
                elif NumDataAug==2:
                    imgAfterAug = dataAug.blur(img)

                imgAfterAugPath = os.path.join(output_imgSequence_dir, os.path.basename(image_path))
                imgAfterAug.save(imgAfterAugPath)

                mask_path = os.path.join(folder_masks_path, os.path.basename(image_path))
                mask = Image.open(mask_path)

                if NumDataAug==0:
                    MaskAfterAug = dataAug.horizontal_flipping(mask)
                else: 
                    MaskAfterAug = mask
                
                MaskAfterAugPath = os.path.join(output_mask_dir, os.path.basename(mask_path))
                MaskAfterAug.save(MaskAfterAugPath)


Folder created at: trainsetRNN/truthWithoutSequences\BrightnessChange\videoPrueba5
Folder created at: trainsetRNN/image_copy\BrightnessChange\videoPrueba5
Folder created at: trainsetRNN/truthWithoutSequences\Blurred\videoPrueba5
Folder created at: trainsetRNN/image_copy\Blurred\videoPrueba5
Folder created at: trainsetRNN/truthWithoutSequences\Flipped\videoPrueba5
Folder created at: trainsetRNN/image_copy\Flipped\videoPrueba5
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exists.
Folder already exis